In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
import random
import tensorflow.keras.utils as utils
import pydot
from tensorflow.keras import regularizers

In [2]:
dataset_dir = "Datasets/houseprices_ready.csv"

In [3]:
df = pd.read_csv(dataset_dir, index_col=0)
df = df.fillna(df.mean())
df.head(10)

,AboveMedianPrice,LotArea,OverallQual,OverallCond,TotalBsmtSF,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea
0,1,8450,7,5,856,2,1,3,8,0,548
1,1,9600,6,8,1262,2,0,3,6,1,460
2,1,11250,7,5,920,2,1,3,6,1,608
3,0,9550,7,5,756,1,0,3,7,1,642
4,1,14260,8,5,1145,2,1,4,9,1,836
5,0,14115,5,5,796,1,1,1,5,0,480
6,1,10084,8,5,1686,2,0,3,7,1,636
7,1,10382,7,6,1107,2,1,3,7,2,484
8,0,6120,7,5,952,2,0,2,8,2,468
9,0,7420,5,6,991,1,0,2,5,2,205


In [4]:
df.isnull().values.any()

False

In [5]:
df = df.sample(frac=1, random_state=41)

In [6]:
col_num = len(df.columns)
row_num = len(df.index)
print("Number of columns is: {} and number of rows is: {}".format(col_num, row_num))

Number of columns is: 11 and number of rows is: 1460


In [7]:
trainx = df.iloc[:int(0.8*row_num), 1:(col_num)]
trainy = df.iloc[:int(0.8*row_num), 0]

testx = df.iloc[int(0.8*row_num):, 1:(col_num)]
testy = df.iloc[int(0.8*row_num):, 0]

In [8]:
trainy

335     1
1074    1
933     1
982     0
921     0
       ..
1018    0
1016    1
307     0
1450    0
17      0
Name: AboveMedianPrice, Length: 1168, dtype: int64

In [9]:
scaler = StandardScaler()
trainx_scaled = pd.DataFrame(scaler.fit_transform(trainx), columns = trainx.columns, index = trainx.index)
textx_scaled = pd.DataFrame(scaler.transform(testx), columns = testx.columns, index = testx.index)

In [10]:
trainx_scaled

,LotArea,OverallQual,OverallCond,TotalBsmtSF,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea
335,14.391355,-0.810781,0.354888,1.012200,0.770659,-0.756619,0.157079,0.276255,2.126222,0.249948
1074,-0.184336,0.646130,-0.536529,0.427102,0.770659,-0.756619,-1.066042,-0.943654,-0.970321,1.638311
933,-0.190779,0.646130,-0.536529,1.014459,0.770659,-0.756619,0.157079,-0.333699,-0.970321,0.441608
982,-0.686115,0.646130,-0.536529,0.485838,0.770659,-0.756619,-1.066042,-0.333699,0.577951,-0.409173
921,-0.163701,-0.810781,1.246305,0.499392,0.770659,3.188607,1.380199,1.496164,-0.970321,-2.222927
...,...,...,...,...,...,...,...,...,...,...
1018,0.023696,0.646130,-0.536529,-1.506656,0.770659,1.215994,0.157079,0.276255,0.577951,-0.343729
1016,0.126311,0.646130,-0.536529,1.023495,0.770659,-0.756619,0.157079,-0.333699,0.577951,0.011542
307,-0.243721,-0.082326,1.246305,-1.653495,-1.047785,-0.756619,0.157079,-0.333699,-0.970321,-2.222927
1450,-0.142879,-0.810781,-0.536529,-0.350016,0.770659,3.188607,1.380199,0.886209,-0.970321,-2.222927


In [75]:
import random
def create_rand_list(max_val, count):
    randomlist = random.sample(range(0, max_val + 1), count)
    return randomlist

In [11]:
def get_MICS_model(inp_size, drop_out, hidden_num = 4, hidden_size=32):
    inputs = keras.layers.Input(shape=(inp_size), name="input")
        
    h = keras.layers.Dense(hidden_size, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(inputs)
    h = keras.layers.Dropout(drop_out)(h)
    for hidden in range(hidden_num):
        h = keras.layers.Dense(hidden_size, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(h)
        h = keras.layers.Dropout(drop_out)(h) 

    outputs = keras.layers.Dense(1, activation="sigmoid")(h)    
    return keras.Model(inputs=[inputs], outputs = outputs)

In [12]:
trainx_scaled

,LotArea,OverallQual,OverallCond,TotalBsmtSF,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea
335,14.391355,-0.810781,0.354888,1.012200,0.770659,-0.756619,0.157079,0.276255,2.126222,0.249948
1074,-0.184336,0.646130,-0.536529,0.427102,0.770659,-0.756619,-1.066042,-0.943654,-0.970321,1.638311
933,-0.190779,0.646130,-0.536529,1.014459,0.770659,-0.756619,0.157079,-0.333699,-0.970321,0.441608
982,-0.686115,0.646130,-0.536529,0.485838,0.770659,-0.756619,-1.066042,-0.333699,0.577951,-0.409173
921,-0.163701,-0.810781,1.246305,0.499392,0.770659,3.188607,1.380199,1.496164,-0.970321,-2.222927
...,...,...,...,...,...,...,...,...,...,...
1018,0.023696,0.646130,-0.536529,-1.506656,0.770659,1.215994,0.157079,0.276255,0.577951,-0.343729
1016,0.126311,0.646130,-0.536529,1.023495,0.770659,-0.756619,0.157079,-0.333699,0.577951,0.011542
307,-0.243721,-0.082326,1.246305,-1.653495,-1.047785,-0.756619,0.157079,-0.333699,-0.970321,-2.222927
1450,-0.142879,-0.810781,-0.536529,-0.350016,0.770659,3.188607,1.380199,0.886209,-0.970321,-2.222927


In [13]:
inp_size = len(test_x_rand.columns)

NameError: name 'test_x_rand' is not defined

In [14]:
min_losses = {12:[], 14:[], 16:[], 18:[], 20:[], 22:[], 24:[]}

In [16]:
inp_size = len(trainx.columns)
MICS_model = get_MICS_model(inp_size, drop_out = 0.25)
callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50), 
        keras.callbacks.ReduceLROnPlateau("val_loss", factor = 0.8, patience=30,
                                         verbose = 2, mode = "auto", 
                                          min_lr = 1e-6)]
MICS_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss=keras.losses.BinaryCrossentropy(), metrics=["accuracy"])
history = MICS_model.fit(x = [trainx_scaled], y = trainy.values,  
                         validation_data = ([textx_scaled], testy.values),
                         epochs=300, batch_size = 32, callbacks=callback)
training_val_accuracy = history.history["val_accuracy"]
best_row_index = np.argmax(training_val_accuracy)
best_val_accuracy = training_val_accuracy[best_row_index]
best_val_accuracy
#min_losses3[c].append(best_val_loss)
#i = i + 1

Epoch 1/300
37/37 [==============================] - 0s 5ms/step - loss: 0.6307 - accuracy: 0.7748 - val_loss: 0.4052 - val_accuracy: 0.8938
Epoch 2/300
37/37 [==============================] - 0s 2ms/step - loss: 0.4487 - accuracy: 0.8904 - val_loss: 0.3264 - val_accuracy: 0.9007
Epoch 3/300
37/37 [==============================] - 0s 1ms/step - loss: 0.3989 - accuracy: 0.8973 - val_loss: 0.3258 - val_accuracy: 0.9007
Epoch 4/300
37/37 [==============================] - 0s 2ms/step - loss: 0.3713 - accuracy: 0.8964 - val_loss: 0.3451 - val_accuracy: 0.8973
Epoch 5/300
37/37 [==============================] - 0s 1ms/step - loss: 0.3450 - accuracy: 0.9033 - val_loss: 0.2986 - val_accuracy: 0.9041
Epoch 6/300
37/37 [==============================] - 0s 2ms/step - loss: 0.3428 - accuracy: 0.8998 - val_loss: 0.2894 - val_accuracy: 0.8973
Epoch 7/300
37/37 [==============================] - 0s 2ms/step - loss: 0.3452 - accuracy: 0.9033 - val_loss: 0.2980 - val_accuracy: 0.9007
Epoch 8/300
3

0.9246575236320496